# Awareness Model
---

> JASIRI's awareness is generated through advertisements and promotional efforts.

# 2. Assumptions

* We explicitly consider the decay in retention of afroprenuers due to forgetting.
* Initial awareness level prior to advertising, `A_o` is not subject to decay.
* We exclude awareness generated via word of mouth communication.

In [1]:
# Standard libraries: 
import math

# Analysis and plotting modules
import pandas as pd
# import plotly

In [2]:
# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

In [3]:
initial_state = {
    'At': 0,
}
initial_state

{'At': 0}

* `At` fraction of the target market aware fo jasiri at time t

In [4]:
system_params = {
    'β': [0.2],
    'α': [0.08],
}
system_params

{'β': [0.2], 'α': [0.08]}

* `β` represents the responsiveness of awareness of `JASIRI_PROTOCOL` due to advertising
* `α` represents the effect of no advertising on awareness

In [5]:
def NewJasiriAwarenessFraction(previous_At_state,alpha,beta):
    value = ((1 - math.exp(alpha - beta)) * (1 - previous_At_state)) - previous_At_state
    
    return(value)
    

In [6]:
def s_At(params, substep, state_history, previous_state, policy_input): 
    previous_At_state = previous_state['At']
    alpha = params['α']
    beta = params['β']
    
    return 'At', max(NewJasiriAwarenessFraction(previous_At_state, alpha, beta), 0)

In [7]:
partial_state_update_blocks = [
    {
        'policies': {},
        # State variables
        'variables': {
            # Updated in parallel
            'At': s_At
        }
    }
]

In [8]:
sim_config = config_sim({
    "N": 1,
    "T": range(400),
    "M": system_params
})

In [9]:
from cadCAD import configs
del configs[:] # Clear any prior configs

In [10]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)
configs[-1].__dict__

{'sim_config': {'N': 1,
  'T': range(0, 400),
  'M': {'β': 0.2, 'α': 0.08},
  'subset_id': 0,
  'subset_window': deque([0, None]),
  'simulation_id': 0,
  'run_id': 0},
 'initial_state': {'At': 0},
 'seeds': {},
 'env_processes': {},
 'exogenous_states': {},
 'partial_state_updates': [{'policies': {},
   'variables': {'At': <function __main__.s_At(params, substep, state_history, previous_state, policy_input)>}}],
 'policy_ops': [<function cadCAD.configuration.Experiment.<lambda>(a, b)>],
 'kwargs': {},
 'user_id': 'cadCAD_user',
 'session_id': 'cadCAD_user=0_0',
 'simulation_id': 0,
 'run_id': 0,
 'experiment_id': 0,
 'exp_window': deque([1, 0]),
 'subset_id': 0,
 'subset_window': deque([0, None])}

In [11]:
exec_context = ExecutionContext()

In [12]:
simulation = Executor(exec_context=exec_context, configs=configs)

In [13]:
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (400, 2, 1, 1)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.20s


In [17]:
simulation_result = pd.DataFrame(raw_result)